In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import time
import os
import multiprocessing as mp # for speeding up some process
import sys
sys.path.append('../py_model')
from utils import init_logging
import logging 
from fancyimpute import MICE # for imputing
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
import multiprocessing
CPU_USE_RATE = 0.5

Using TensorFlow backend.
/home/linuxbrew/.linuxbrew/opt/python/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# use prediction of model as target of test

In [29]:
# base features
df = pd.read_hdf('../features/base_featurs.h5','base_featurs')
copy_for_the_following_merge = df[['SK_ID_CURR','TARGET']].copy()
no_need_to_inpute = ['index']
df.drop(no_need_to_inpute, axis = 1, inplace = True)
print ('df', df.shape)
# find discrete features
discrete_features = []
continuous_features = []
for f in df.columns.tolist()[:]:
    if df[f].value_counts().size < 20:
        discrete_features.append(f)
    else:
        continuous_features.append(f)
print (len(discrete_features), len(continuous_features))
# check 
'SK_ID_CURR' in discrete_features, 'TARGET' in discrete_features, 'index' in discrete_features

df (356251, 280)
72 208


(False, True, False)

In [6]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
from operator import itemgetter

# setting
CPU_USE_RATE = 0.7
num_folds = 5
# input
#df = df[discrete_features]
print ('df',df.shape)
train_df = df[df['TARGET'].notnull()]
print ('train', train_df.shape)
test_df = df[df['TARGET'].isnull()]
print ('test', test_df.shape)
# output
oof_preds = np.zeros(train_df.shape[0]) # substitue the target of training part in df
print ('oof_preds', oof_preds.shape)
train_preds = np.zeros(train_df.shape[0])
print ('train_preds', train_preds.shape)
# test
sub_preds = np.zeros(test_df.shape[0]) # substitue the target of testing part in df
print ('sub_preds', sub_preds.shape)
feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
# data peeking
folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=int(time.time()))
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
    s = time.time()
    print ('n_fold', n_fold)
    train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
    print ('train_x',train_x.shape)
    #print ('train_idx',train_idx)
    valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]
    print ('valid_x',valid_x.shape)
    #print ('valid_idx',valid_idx)
    ratio = pd.Series(train_y).value_counts().iloc[0] / pd.Series(train_y).value_counts().iloc[1] # > 1
    print ('ratio', ratio)
    #----------------
    # model 
    #----------------
    clf = LGBMClassifier(
        nthread=int(multiprocessing.cpu_count()*CPU_USE_RATE),
        n_estimators=1,
        learning_rate=0.02,
        num_leaves=34, # 20
        colsample_bytree=0.2, #0.9497036 < 0.2
        subsample=0.8715623,
        max_depth=8, # 7
        reg_alpha=0.041545473, # 0.3
        reg_lambda=0.0735294,
        min_split_gain=0.0222415,
        min_child_weight=39.3259775, # 60
        silent=-1,
        verbose=-1,
        scale_pos_weight = ratio,
        random_state = int(time.time()),
        )
    #----------------
    # training 
    #----------------
    clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
        eval_metric= 'auc', verbose= True, early_stopping_rounds= 1) # early_stopping_rounds= 100
    # training/validating
    probas_ = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
    print ('probas_', probas_)
    fpr, tpr, thresholds = roc_curve(valid_y, probas_)
    exp = []
    for th in thresholds:
        y_scores = [1 if p_ >= th else 0 for p_ in probas_]
        exp.append((th, roc_auc_score(valid_y, y_scores)))
    best_th = max(exp, key = itemgetter(1))[0]
    best_performance = max(exp, key = itemgetter(1))[1]
    print ('best_th', best_th)
    print ('best_auc', best_performance) # 0.613309518754671
    oof_preds[valid_idx] = [1 if p_ >= best_th else 0 for p_ in probas_]
    print ('oof_preds', pd.Series(oof_preds[valid_idx]).value_counts()) 
    # testing
    test_prob_ = clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1]
    sub_preds +=  np.array([1 if p_ >= best_th else 0 for p_ in test_prob_])/ folds.n_splits
    e = time.time()
    print (e - s)
  

df (356251, 280)
train (307507, 280)
test (48744, 280)
oof_preds (307507,)
train_preds (307507,)
sub_preds (48744,)
n_fold 0
train_x (246005, 278)
valid_x (61502, 278)
ratio 11.386958710976838
[1]	training's auc: 0.691036	valid_1's auc: 0.677381
Training until validation scores don't improve for 1 rounds.
Did not meet early stopping. Best iteration is:
[1]	training's auc: 0.691036	valid_1's auc: 0.677381
probas_ [0.09394466 0.08918081 0.08971504 ... 0.08971504 0.0908286  0.08514889]
best_th 0.08971503673201754
best_auc 0.6332488090172428
oof_preds 0.0    38107
1.0    23395
dtype: int64
50.50724768638611
n_fold 1
train_x (246005, 278)
valid_x (61502, 278)
ratio 11.386958710976838
[1]	training's auc: 0.733938	valid_1's auc: 0.731145
Training until validation scores don't improve for 1 rounds.
Did not meet early stopping. Best iteration is:
[1]	training's auc: 0.733938	valid_1's auc: 0.731145
probas_ [0.08973389 0.09012783 0.08315972 ... 0.08660937 0.08973389 0.08728378]
best_th 0.0897338

In [7]:
oof_preds.shape

(307507,)

In [8]:
pd.Series(oof_preds).value_counts()

0.0    189804
1.0    117703
dtype: int64

In [9]:
sub_preds.shape

(48744,)

In [10]:
pd.Series(sub_preds).value_counts()

0.0    18157
1.0     7371
0.2     7144
0.8     5432
0.4     5341
0.6     5299
dtype: int64

In [11]:
pd.Series([1 if i >= 0.5 else 0 for i in sub_preds]).value_counts() # [0:0, 1/5:0, 2/5:0, 3/5: 1,..]

0    30642
1    18102
dtype: int64

In [ ]:
# merge and output for imputing
copy_for_the_following_merge.TARGET.value_counts().iloc[0] / copy_for_the_following_merge.TARGET.value_counts().iloc[1]

# step1: target = oof_preds + sub_preds
# step2: 把他concate with df 

In [22]:
sub_preds = np.array([1 if i >= 0.5 else 0 for i in sub_preds])
target = np.concatenate((oof_preds, sub_preds), axis = 0)
target.shape 

(356251,)

In [30]:
df = df[['SK_ID_CURR'] + discrete_features]
df['TARGET'] = target

In [31]:
df.shape

(356251, 73)

In [32]:
'SK_ID_CURR' in df.columns.tolist(), 'TARGET' in df.columns.tolist(), 'index' in df.columns.tolist()

(True, True, False)

In [33]:
df[['SK_ID_CURR', 'TARGET']]

,SK_ID_CURR,TARGET
0,100002,1.0
1,100003,1.0
2,100004,0.0
3,100006,0.0
4,100007,1.0
5,100008,0.0
6,100009,0.0
7,100010,0.0
8,100011,1.0
9,100012,1.0
